# Project Group - 

Members: Jarrik Algera, Lorenzo Bouman, Jasper IJsselstein, Joep Neelissen, Tjerk Rijkens


Student numbers: 4673247, 1234567

# Research Objective

*Requires data modeling and quantitative research in Transport, Infrastructure & Logistics*

The objective of this project is to examine the relationship between weather conditions and train disruptions in the Netherlands. Railway disruption records are combined with meteorological data to quantify the extent to which factors such as precipitation, wind, and temperature are associated with delays and service interruptions. The analysis aims to establish whether specific weather conditions systematically affect railway performance and to what degree.

## Main question

#### *To what extent do weather conditions influence train disruptions in the Netherlands?*

### Sub-questions

*Are disruptions more frequent on days with adverse weather compared to days with mild weather?*


*Do disruption durations increase under heavy precipitation, strong wind, or temperature extremes?*


*Which weather variable shows the strongest association with disruption frequency?*


*Are there seasonal differences in how weather affects train disruptions?*


*Are there regional variations in the relationship between weather and disruptions?*

# Contribution Statement

*Be specific. Some of the tasks can be coding (expect everyone to do this), background research, conceptualisation, visualisation, data analysis, data modelling*

**Author 1**:

**Author 2**:

**Author 3**:

# Data Used

## Datasets

Two primary datasets are available for this study. The first dataset, disruptions-2024(2).csv, contains records of train disruptions in the Netherlands during 2024. Each record provides details on the type of disruption, its duration, and the time of occurrence. This dataset represents the dependent variable, as it directly reflects railway system performance.

The second dataset, vorigemaand.txt, is derived from the Royal Netherlands Meteorological Institute (KNMI) and provides weather observations for the same period. It includes variables such as precipitation intensity, temperature, and wind speed. These variables serve as independent factors to explain variation in railway disruptions.

# Data Pipeline

## Intended Data Analysis Pipeline

### The planned analysis will follow several stages:

Data cleaning and preparation

Align disruption data with corresponding weather observations based on time and location.

Address missing values and filter irrelevant records.

Exploratory analysis

Compute descriptive statistics for disruption frequency and duration.

Visualize weather variables alongside disruptions to detect initial patterns.

Correlation analysis

Assess associations between weather conditions and disruption frequency or duration.

Modeling

Apply regression models to estimate the effect of precipitation, wind, and temperature on disruption likelihood and duration.

Investigate seasonal and regional variation in the relationship between weather and disruptions.

Evaluation and visualization

Present results using time series plots, scatterplots, and heatmaps, alongside statistical measures.